# eCREST_notebook

Basic functions for reconstructing cells.

# Setup

Do the following two setup steps regardless of how you will be using this script. 

## 1. Imports

Run the following code cell to import the necessary packages and modules. 

In [ ]:
############################################################################################################################ 
# Get the latest CREST files for each ID within the target folder (dirname)

from pathlib import Path
import json
from sqlite3 import connect as sqlite3_connect
from sqlite3 import DatabaseError
from igraph import Graph as ig_Graph
from igraph import plot as ig_plot
from scipy.spatial.distance import cdist
from random import choice as random_choice
from itertools import combinations
from numpy import array, unravel_index, argmin, mean,unique,nan
import pandas as pd
from copy import deepcopy
from datetime import datetime
from time import time
import neuroglancer
from webbrowser import open as wb_open
from webbrowser import open_new as wb_open_new
import neuroglancer

# from eCREST_cli_beta import ecrest, import_settings
from eCREST_cli import ecrest, import_settings, get_cell_filepaths

The 'ecrest' class has been imported from eCREST_cli.py

An instance of this object will be able to do things like:
- open an neuroglancer viewer for proofrieading (see "Proofread using CREST")
    - add-remove segments (using graph feature for efficiency)
    - format itself and save itself as a CREST-style .json
- add or remove annotation layers (see "Annotation Layers")
- check for overlap with other .json files in a directory folder (see "check for overlap")
- label cell structures
- add base_segments from a list (see "add segments")
- import annotations from another file (see "Annotation Import")
- convert from neuroglancer json (see "Convert From Neuroglancer to eCREST")
    - format itself and save itself as a CREST-style .json
    


## 2. Settings definitions

Whether you are converting from neuroglancer or creating a new reconstruction, the settings_dict parameters is needed to create CREST json files with correct formatting. 
- 'save_dir' : the directory where JSON files are saved 
- 'cred' and 'db_path' : specify the path to the agglomeration database file on your local computer. 

In [ ]:
path_to_settings_json = '/Users/kperks/Documents/ell-connectome/eCREST-local-files/settings_dict.json'
settings_dict = import_settings(path_to_settings_json)

# Proofread using (e)CREST

The ```ecrest``` class defined in eCREST_cli.py can be used to proofread base_segment reconstructions enhanced by the agglomeration database.

An instance of this class can be initialized with either:
- ecrest(segment_id): a "main_base_id" in *int* format
- ecrest(filepath): an existing CREST .json file
- ecrest(segment_id, segment_list): the main_base_id from the neuroglancer file you are converting and a list of base_segments.

The ```launch_viewer``` flag default is "False" so that you can interact with the contents of a reconstruction without actually opening it visually in a neuroglancer tab. **NOTE**: Some ecrest functions require that the ecrest instance is created with ```launch_viewer==True```.

## NEW reconstruction from segment ID

If you wanted to start reconstructing a new cell from a main base segment, you would use the following code block to launch.

To change the save location you can specify the ```directory_path``` flag in the ```save_cell_graph()``` module

In [ ]:
segment_id = 476924620
crest = ecrest(settings_dict,segment_id = segment_id, launch_viewer=True)

In [ ]:
# SAVE YOUR WORK!

crest.save_cell_graph() # Default location is Path(settings_dict['save_dir'])

## EDIT reconstruction from file

If you wanted to edit a reconstruction from an existing file, you would use the following code block to launch.

Specify the cell_id and the path to the directory that cell is in. 

> NOTE: You can also directly copy paste the full filepath to the cell you want to open and pass it to the ```filepath``` flag.  
In that case, the only code you need is crest = ecrest(settings_dict,filepath= [*paste filepath here*], launch_viewer=True)

To change the save location you can specify the ```directory_path``` flag in the ```save_cell_graph()``` module

> To overwrite a file (not recommended), specify ```(directory_path = cell_filepaths[cell_id].parent, file_name = cell_filepaths[cell_id].name)```

In [ ]:
directory_path = Path(settings_dict['save_dir']) / 'todo_presynaptic/lf_393325331'  ## specify the directory path
cell_filepaths = get_cell_filepaths(directory_path) # gets filepaths for all cells in a directory

In [ ]:
cell_id = '644143297' # specify the cell id

crest = ecrest(settings_dict,filepath= cell_filepaths[cell_id], launch_viewer=True)

crest.change_key_binding({"alt+mousedown0" : "add-or-remove-seg"})

## Define Cell type

The cell type strings to use are:  
aff, grc-d, grc-s, sgx1, sgx2, sg1, sg2, mg1, mg2, lf, lg, uk, fov

You can also check the current cell type assigned by using ```crest.get_ctype("manual")```

In [ ]:
cell_type = 'grc-d' # Assign the cell type then run the code cell

crest.define_ctype(cell_type,"manual")

## Change Function Keybindings

speficy any keybindings that you want in ```keybindings_dict``` and pass that to the class module ```change_key_binding```. The keybinding mapping will be returned. 

In [ ]:
keybindings_dict = {
    "alt+mousedown0" : "add-or-remove-seg"
}

crest.change_key_binding(keybindings_dict)

## Check for overlap with other .json files in a directory

Use the ecrest module ```get_base_segments_dict``` to get a dictionary of {'cell_id' : 'base_segments'} for all .json files in a folder.  
Then use the ecrest module ```check_duplicates``` to get a dataframe of any instances of overlap between that cell and the .json files from the folder.

*Tips*:
- You do ***not*** need to run ```get_base_segments_dict``` every time. For each new crest instance you create, you can just skip to the ```check_duplicates``` step.
- You can create multiple dictionaries to check against... you can create a separate line of code for each. 

In [ ]:
base_segments_net = crest.get_base_segments_dict(Path(settings_dict['save_dir']))
base_segments_todo = crest.get_base_segments_dict(Path(settings_dict['save_dir']) / 'kp/392042360_grc-s_pre')

In [ ]:
print('overlap in main network:'); df = crest.check_duplicates(base_segments_net); display(df)
print('overlap in todo folder:'); df = crest.check_duplicates(base_segments_todo); display(df)

## Add/Remove Annotation layers

Because of how CREST saves the .json state, annotation layers need to be added/removed programatically rather than via the neuroglancer viewer directly.

Comment/uncomment the following two module implementations as needed.

In [ ]:
crest.add_endpoint_annotation_layers(['soma'])

# crest.del_endpoint_annotation_layers(['soma'])

## SAVE YOUR WORK!

In [ ]:
crest.save_cell_graph() # Default location is Path(settings_dict['save_dir'])

# Other

In [ ]:
# crest.add_endpoint_annotation_layers(['soma'])

In [ ]:
base_segments_net = crest.get_base_segments_dict(Path(settings_dict['save_dir']))
base_segments_todo = crest.get_base_segments_dict(Path(settings_dict['save_dir']) / 'todo_presynaptic/lf_393325331')

In [ ]:
print('overlap in main network:'); df = crest.check_duplicates(base_segments_net); display(df)
print('overlap in todo folder:'); df = crest.check_duplicates(base_segments_todo); display(df)

In [ ]:
cell_type = 'sgx1' # Assign the cell type then run the code cell

crest.define_ctype(cell_type,"manual")

In [ ]:
# SAVE YOUR WORK!

crest.save_cell_graph() # Default location is Path(settings_dict['save_dir'])

In [ ]:
# cell_filepaths[cell_id].name

In [ ]:
# crest.save_cell_graph(directory_path = cell_filepaths[cell_id].parent, file_name = cell_filepaths[cell_id].name)

In [ ]:
# crest.get_ctype('manual')